In [1]:
### parameters
place = 'tel_aviv'
feature = 'road_right'

In [2]:
import os
import zipfile
from glob import glob
import numpy as np


import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union, nearest_points
from shapely.geometry import Point,Polygon,MultiPolygon

import networkx as nx

# Get the current working directory (e.g., the folder you're running from)
from pathlib import Path
from scipy.ndimage import generic_filter
import warnings
warnings.filterwarnings(action='ignore')
crs_prj = 'EPSG:2039'

cwd = Path().resolve()
# Get the parent directory
parent_folder = f'{cwd.parent}/places/{place}'
data_folder = f'{parent_folder}/shp'
os.makedirs(f'{parent_folder}',exist_ok=True)
os.makedirs(f'{parent_folder}/shp',exist_ok=True)
os.makedirs(f'{parent_folder}/shp/{feature}',exist_ok=True)
detail_folder = f'{data_folder}/{feature}'

In [3]:

# Define paths
input_dir = f'{detail_folder}/raw_data' # Directory containing zip files
output_dir = f'{detail_folder}/unzip_raw_data'  # Directory to store outputs
os.makedirs(output_dir, exist_ok=True)

# Temporary directory for unzipping
temp_dir = os.path.join(output_dir, "unzipped")
os.makedirs(temp_dir, exist_ok=True)

# Initialize list to hold filtered GeoDataFrames
filtered_gdfs = []

In [33]:
cols_to_save = ['oidmigrash', 'msgush', 'msmigrash', 'idtaba', 'tyeudkarka', 'tyeudrashi', 'msshetach', 'geometry']

# Unzip and process each shapefile
for zip_path in glob(os.path.join(input_dir, "*.zip")):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        extract_path = os.path.join(temp_dir, os.path.splitext(os.path.basename(zip_path))[0])
        zip_ref.extractall(extract_path)

        for shp_path in glob(os.path.join(extract_path, "**", "*.shp"), recursive=True):
            try:
                gdf = gpd.read_file(shp_path)
                if not gdf.empty:
                    filtered_gdfs.append(gdf)
            except Exception as e:
                print(f"Failed to process {shp_path}: {e}")

# Merge, filter, and save
if filtered_gdfs:
    merged_gdf = gpd.GeoDataFrame(pd.concat(filtered_gdfs, ignore_index=True), crs=filtered_gdfs[0].crs)
    merged_gdf = merged_gdf[cols_to_save]

    merged_gdf.to_file(os.path.join(output_dir, "merged_gdf.shp"))

    # Filter for "תחבורה" in tyeudrashi
    filtered = merged_gdf[merged_gdf["tyeudrashi"] == "תחבורה"]
    filtered.to_file(os.path.join(output_dir, "tyeudrashi.shp"))

    # Remove duplicates by "oidmigrash"
    filtered_unique = filtered.drop_duplicates(subset="oidmigrash")
    final_path = os.path.join(output_dir, "merged_filtered_unique.shp")
    filtered_unique.to_file(final_path)

    print(f"✅ Final shapefile saved to: {final_path}")
else:
    print("⚠️ No matching shapefiles found.")


In [6]:
# # Creating of perpendicular lines at specific positions along each street segment — typically at 1/3, 1/2, and 2/3 of the segment — on both the left and right sides.
# 
# ✅ Purpose:
# To sample the cross-section of the street at multiple representative points.
# 
# These perpendiculars simulate measuring from the road center outward toward the edge (e.g., curb or parcel boundary).
# Load your polygon layer
gdf = filtered.copy()
merged_geom = unary_union(gdf.geometry)
# Split multipolygon into separate rows
if isinstance(merged_geom, MultiPolygon):
    geometries = list(merged_geom.geoms)
else:
    geometries = [merged_geom]

# Create a new GeoDataFrame with each part as a separate row
final_gdf = gpd.GeoDataFrame(geometry=geometries, crs=gdf.crs)
final_gdf.to_file(f"{detail_folder}/dissolved_cleaned.shp")
# Convert each polygon to its boundary (LineString or MultiLineString)
gdf_polyline = final_gdf.copy()
gdf_polyline["geometry"] = gdf_polyline["geometry"].boundary
# # Optional: Save to file
# gdf_polyline.to_file(f"{detail_folder}/polygon_boundaries_as_lines.shp")
lines_gdf= gpd.read_file(f'{detail_folder}/side_lines_gdf0.shp')

# Extract the start point of each line
lines_gdf["start_point"] = lines_gdf.geometry.apply(lambda geom: Point(geom.coords[0]))

# Convert to GeoDataFrame of points for spatial join
start_points_gdf = gpd.GeoDataFrame(lines_gdf.drop(columns="geometry"), geometry=lines_gdf["start_point"], crs=lines_gdf.crs)

# Spatial join: find start points within road_right polygons
joined = gpd.sjoin(start_points_gdf, final_gdf, how="inner", predicate="within")

# Use index to filter original lines
filtered_lines = lines_gdf.loc[joined.index]


# Save result
filtered_lines.drop(columns=["start_point"]).to_file(f"{detail_folder}/filtered_perpendicular_lines.shp")

In [34]:
#  Computing the intersection of each perpendicular line with street boundary polylines (converted from polygons), and measuring the distance from the line’s start point to the closest intersection.
# 
# ✅ Purpose:
# To determine the distance from the center of the road to its edge on one side — this gives one half of the street width.
# 
# If done for both left and right sides, you can calculate the full right-of-way width.

# Build spatial index on street edge polylines
edge_sindex = gdf_polyline.sindex

# Prepare list to collect results
results = []

# Process each perpendicular line
for idx, row in filtered_lines.iterrows():
    line = row.geometry
    start_point = row["start_point"]

    # Use spatial index to find candidate polylines near the line
    candidate_idx = list(edge_sindex.intersection(line.bounds))
    candidate_edges = gdf_polyline.iloc[candidate_idx]

    # Compute intersections between the perpendicular line and candidate edges
    intersections = []
    for _, edge_row in candidate_edges.iterrows():
        intersection = line.intersection(edge_row.geometry)
        if not intersection.is_empty:
            if intersection.geom_type == "Point":
                intersections.append(intersection)
            elif intersection.geom_type.startswith("Multi"):
                intersections.extend([
                    geom for geom in intersection.geoms if geom.geom_type == "Point"
                ])

    # Keep the closest intersection point to the start of the line
    if intersections:
        nearest_point = min(intersections, key=lambda pt: start_point.distance(pt))
        results.append({
            **row.to_dict(),
            "intersection_point": nearest_point,
            "width_side": start_point.distance(nearest_point)
        })

# Create final GeoDataFrame with intersection points
results_gdf = gpd.GeoDataFrame(results, geometry="intersection_point", crs=crs_prj)

# Optional: Save the result
# results_gdf.to_file(f"{detail_folder}/efficient_width_side_result.shp")



In [36]:

# Compute representative left, right, and total road widths for each street segment by aggregating perpendicular width measurements and merging them with street geometries.
# Step 1: Merge perpendicular line results (width_side) into filtered_lines by 'oidrechov' and 'side'
width_df = filtered_lines.merge(
    results_gdf[['oidrechov', 'side', 'width_side']],
    on=['oidrechov', 'side'],
    how='left'
)

# Step 2: Split 'side' into 'position' and 'onlyside' (e.g., 'start_left' → position='start', onlyside='left')
width_df[['position', 'onlyside']] = width_df['side'].str.extract(r'(\w+)_(\w+)', expand=True)

# Step 3: Load street geometries
streets = gpd.read_file(f'{data_folder}/streets.shp')[['oidrechov', 'geometry']]

# Step 4: Define function to calculate representative width
def summarize_width(values):
    values = [v for v in values if pd.notnull(v)]
    if len(values) == 0:
        return np.nan
    elif len(values) == 1:
        return values[0]
    elif len(values) == 2:
        return np.mean(values)
    elif len(values) == 3:
        pairs = [(values[i], values[j]) for i in range(3) for j in range(i + 1, 3)]
        closest_pair = min(pairs, key=lambda pair: abs(pair[0] - pair[1]))
        return np.mean(closest_pair)
    else:
        return np.nan

# Step 5: Group by street and side, and summarize width
summary = width_df.groupby(['oidrechov', 'onlyside'])['width_side'].apply(summarize_width).unstack()

# Step 6: Rename columns and calculate total width
summary = summary.rename(columns={"left": "left", "right": "right"})
summary["road_right"] = summary["left"] + summary["right"]
summary.reset_index(inplace=True)

# Step 7: Merge width estimates into street geometries
street_edges_gdf = streets.merge(summary, on='oidrechov', how='left')

# Step 8: Save final output
street_edges_gdf.to_file(f'{detail_folder}/{feature}.shp')
